# Scraping Data

In [81]:
from tableauscraper import TableauScraper as TS

url = "https://public.tableau.com/views/GatorEvalsFall2019toFall2021PublicData/GatorEvalsPublic"

ts = TS()
ts.loads(url)
workbook = ts.getWorkbook()

worksheet = workbook.getWorksheet("GatorEvals Public Data")
data = worksheet.data
head = data.head()

print("Loaded GatorEvals Public Dataset!")


Loaded GatorEvals Public Dataset!


In [82]:
all_filters = worksheet.getFilters()
college_options = all_filters[0]["values"]
course_options = all_filters[1]["values"]
department_options = all_filters[2]["values"]
instructor_options = all_filters[3]["values"]
term_options = all_filters[4]["values"]
print("FILTER OPTIONS:")
print("---------------")
for f in all_filters:
    print(f["column"])

FILTER OPTIONS:
---------------
COLLEGE
COMBINED_COURSE
DEPARTMENT
INSTRUCTOR_NAME
Term


In [83]:
from tqdm import tqdm
import pandas as pd

# Takes about 30 hours to run 😬

overall_columns = ["course_num", "professor", "term", "question_num", "rating", "percentage"]
overall_dataset = pd.DataFrame(columns=overall_columns)

question_texts = data["University Core Questions-value"].truncate(0, 9).to_list()
question_dict = {}
for i in range(0, len(question_texts)):
    question_dict[question_texts[i]] = i

professor_list = []

filters = all_filters

for j in range(0, len(department_options)):
    department = department_options[j]

    ts.loads(url)
    workbook = ts.getWorkbook()
    worksheet = workbook.getWorksheet("GatorEvals Public Data")
    
    filtered_workbook = worksheet.setFilter("DEPARTMENT", department)
    filtered_worksheet = filtered_workbook.getWorksheet("GatorEvals Public Data")
    filters = filtered_worksheet.getFilters()
    courses = filters[1]["values"]
    course_progress_desc = f"Progress through courses in {department}"

    overall_dataset = pd.DataFrame(columns=overall_columns)

    for course in tqdm(courses, desc=course_progress_desc):
        ts.loads(url)
        workbook = ts.getWorkbook()
        worksheet = workbook.getWorksheet("GatorEvals Public Data")

        filtered_workbook = worksheet.setFilter("DEPARTMENT", department)
        filtered_worksheet = filtered_workbook.getWorksheet("GatorEvals Public Data")
        filtered_workbook = filtered_worksheet.setFilter("COMBINED_COURSE", course)
        filtered_worksheet = filtered_workbook.getWorksheet("GatorEvals Public Data")
        filters = filtered_worksheet.getFilters()
        professors =  filters[3]["values"]
        for professor in professors:
            ts.loads(url)
            workbook = ts.getWorkbook()
            worksheet = workbook.getWorksheet("GatorEvals Public Data")

            filtered_workbook = worksheet.setFilter("DEPARTMENT", department)
            filtered_worksheet = filtered_workbook.getWorksheet("GatorEvals Public Data")
            filtered_workbook = filtered_worksheet.setFilter("COMBINED_COURSE", course)
            filtered_worksheet = filtered_workbook.getWorksheet("GatorEvals Public Data")
            filtered_workbook = filtered_worksheet.setFilter("INSTRUCTOR_NAME", professor)
            filtered_worksheet = filtered_workbook.getWorksheet("GatorEvals Public Data")
            filters = filtered_worksheet.getFilters()
            terms =  filters[4]["values"]

            for term in terms:
                filtered_workbook = filtered_worksheet.setFilter("Term", term)
                filtered_worksheet = filtered_workbook.getWorksheet("GatorEvals Public Data")

                try:
                    current_data = filtered_worksheet.data[["University Core Questions-alias", "AVG(RESPONSE_VALUE)-alias", "CNT(RESPONSE_VALUE)-alias"]]
                    current_data.columns = ["question", "rating", "percentage"]

                    current_data.insert(0, "course_num", [course] * len(current_data.index))
                    current_data.insert(1, "professor", [professor] * len(current_data.index))
                    current_data.insert(2, "term", [term] * len(current_data.index))
                    question_numbers = []
                    for i in range(0, len(current_data.index)):
                        question_numbers.append(question_dict[current_data.loc[i, "question"]])
                    
                    current_data.insert(3, "question_num", question_numbers)

                    current_data.drop("question", axis=1, inplace=True)

                    overall_dataset = pd.concat([overall_dataset, current_data])
                except:
                    print(f"COULDN'T GET DATA FOR {course}, by {professor} during {term}")

    output_csv_filename = f"full_scraped_evals_{j}.csv"
    overall_dataset.to_csv(output_csv_filename)

Progress through courses in AGL(AG)-Agricultural & BiolEng:   0%|          | 0/44 [00:01<?, ?it/s]


KeyboardInterrupt: 

# Data Processing

In [2]:
import pandas as pd

# Process data
for i in range(0, 179):
    input_csv_filename = f"original/full_scraped_evals_{i}.csv"
    department_data = pd.read_csv(input_csv_filename)
    department_data.drop(department_data.columns[[0]], axis = 1, inplace = True)
    department_data.insert(0, "department", [i] * len(department_data.index))
    output_csv_filename = f"processed/full_scraped_evals_{i}.csv"
    department_data.to_csv(output_csv_filename)

In [20]:
import os
import pandas as pd

# Concatenate all department csv files into one big one
os.system("cat processed/full_scraped_evals*.csv > processed/combined_data.csv")

overall_dataset = pd.read_csv("processed/combined_data.csv")

# Drop indexing based on department, filter out all header columns that happen
# in the middle of the data.
overall_dataset.drop(overall_dataset.columns[[0]], axis=1, inplace=True)
overall_dataset = overall_dataset[overall_dataset.department != "department"]

# Convert "Term" strings to indices
term_dict = {}
for i in range(0, len(term_options)):
    term_dict[term_options[i]] = i

terms_list = []

for term in overall_dataset["term"]:
    terms_list.append(term_dict[term])

overall_dataset["term"] = terms_list


In [80]:
# Combine all rows that are the same question (in progress)
merged_dataset = overall_dataset.drop(["rating", "percentage"], axis=1)
merged_dataset.drop_duplicates(inplace=True)

current_rating_dataframe = pd.DataFrame()

for i in range (1, 6):
    current_rating_dataframe = overall_dataset[overall_dataset.rating == str(i)]
    current_rating_dataframe.drop("rating", axis=1, inplace=True)
    current_rating_dataframe.columns = ['department', 'course_num', 'professor', 'term', 'question_num', 'percentage_' + str(i)]
    merged_dataset = pd.merge(merged_dataset, current_rating_dataframe, how="left", on=["department", "course_num", "professor", "term", "question_num"])

merged_dataset.fillna(0, inplace=True)

merged_dataset.to_csv("final_data.csv")


/tmp/ipykernel_3243/338862513.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_rating_dataframe.drop("rating", axis=1, inplace=True)
/tmp/ipykernel_3243/338862513.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_rating_dataframe.drop("rating", axis=1, inplace=True)
/tmp/ipykernel_3243/338862513.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_rating_dataframe.drop("rating", axis=1, inplace=True)
/tmp